In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

In [3]:
PATH = Path('/content/data/')

In [4]:
np.random.seed(42)

In [5]:
src = (ImageList.from_csv(PATH,'train.csv',folder='train',suffix='.jpg')
       .split_by_rand_pct(0.2,42)
       .label_from_df()
      )

In [6]:
tfms = get_transforms(flip_vert=True)

In [7]:
def getdata(sz,bs):
    return src.transform(tfms=tfms,size=sz).databunch(bs=bs).normalize(imagenet_stats)

In [8]:
sz = 500
bs = 64

In [9]:
data = getdata(sz,bs)

In [10]:
arch = models.squeezenet1_1

In [11]:
learn = cnn_learner(data,arch,metrics=[accuracy])

In [ ]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy,time
